In [64]:

#Импортируем необходимые для работы библиотеки

import pandas as pd
import pandas.io.sql as psql
import psycopg2

#проверяем, что есть подключение к базе данных.
conn = psycopg2.connect(host = 'localhost',dbname='Course', user='postgres', 
                        password='postgres',port='5432')
cursor = conn.cursor()

In [65]:
#функции для коннекта к БД

#Реазилуем функцию для поиска логина/пароля из файла, что бы явно их в дальнейшем не указывать.

def get_log_pass():
    path = '/home/ab170692gda/credentials.ini'
    try:
        from configparser import ConfigParser
    except ImportError:
        from ConfigParser import ConfigParser  # ver. < 3.0
    
    # Инициализация
    config = ConfigParser()
    
    # Вычитка
    config.read(path)
    
    # Получаем логин и пароль
    user = config.get('db', 'user')
    password = config.get('db', 'password')
    return user, password


#Реализуем функцию соединения с бд
def create_connection(user_name=None, user_pass=None):
    if (user_name is None)&(user_pass is None):
        user_name, user_pass = get_log_pass()

    try:
        connector = psycopg2.connect(
                        host = 'localhost',
                        dbname='Course', 
                        user=user_name, 
                        password=user_pass,
                        port='5432')

        


        return connector
    except Exception as e:
        print('Connect exception: sdfsdf', e)

    return None

#проверяем корректность
a = create_connection()

In [66]:
# 1. понимаем список айди из файла
header_list  = ['TranID_CSV', 'DT_CSV', 'AMT_CSV', 'Comm_CSV'] #Создаем хэдеры(заголовки) колонок
#Вычитываем csv файл
data_csv     = pd.read_csv(f'/home/ab170692gda/TestCompareData.csv'
                          ,sep=','
                          ,decimal="."
                          ,names=header_list
                          ,parse_dates=['DT_CSV']
                           )
#Проверяем корректность.
print(data_csv) 

    TranID_CSV     DT_CSV  AMT_CSV         Comm_CSV
0          124 2021-09-03  2900.52   Get Test Text1
1          954 2021-09-16   737.52  Get Test Text11
2          896 2021-09-20   456.52  Get Test Text10
3          765 2021-08-30  1300.45   Get Test Text9
4          876 2021-09-17   786.52   Get test Text8
5          654 2021-09-18   567.52   Get Test Text7
6          522 2021-09-19  3457.04   Get Test Text6
7          754 2021-09-21  7635.52     Get Test Te5
8          543 2021-09-22   756.52   Get Test Text4
9          431 2021-09-19  4567.52   Get Test Text3
10         421 2021-09-20  4566.02   Get Test Text2


In [67]:
# 2. по этому списку идем в бд, выгружаем с бд в буфер
x = list(data_csv.TranID_CSV)

# из массива преобразуем строку
placeholders=','.join((map(str, x)))
#Указываем запрос с теми айди, которые получили из csv файла
query = "select * from TestCompareData where TRanID in (%s)"% placeholders

#Вызываем запрос в бд используя ранее созданное подключение
conn2= create_connection()
with conn2 as connector:
   data_db= psql.read_sql(query, connector, params=placeholders)

#Проверяем корректность
print(data_db)


    tranid          dt      amt             comm
0      765  2021-08-30  1200.45   Get Test Text9
1      421  2021-09-20  4566.02   Get Test Text2
2      754  2021-09-21  7635.52   Get Test Text5
3      431  2021-09-19  4567.52   Get Test Text3
4      896  2021-09-20   456.52  Get Test Text10
5      543  2021-09-22   756.52   Get Test Text4
6      522  2021-09-19  3457.04   Get Test Text6
7      124  2021-09-21  2900.52   Get Test Text1
8      954  2021-09-22   757.52  Get Test Text11
9      876  2021-09-17   786.52   Get Test Text8
10     654  2021-09-18   567.52   Get Test Text7


In [68]:

#3. сравниваем данные


# "Забираем" айди операции
data_result = data_csv.join(data_db.set_index('tranid'), on='TranID_CSV')

# Указываем толеранс в суммах, пусть будет 2%
tolerance=2

#проставляем изначально по всем суммам True
data_result['AMT_RES'] = True
#Высчитываем разницу, если превышение указанной константы - устанавливаем False(как проверка значений не пройдена) 
data_result.loc[abs(100.-100.*data_result['AMT_CSV'] / data_result['amt'])>=tolerance,'AMT_RES'] = False
#Установка NaN значений как False
data_result.loc[data_result.amt.isnull(),'AMT_RES'] = False

#Сравнивание дат и комментария
data_result['DT_RES'] = data_result['DT_CSV'] == data_result['dt']
data_result['Comm_RES'] = data_result['Comm_CSV'] == data_result['comm']

#Проверка корректности
print(data_result)

    TranID_CSV     DT_CSV  AMT_CSV         Comm_CSV          dt      amt  \
0          124 2021-09-03  2900.52   Get Test Text1  2021-09-21  2900.52   
1          954 2021-09-16   737.52  Get Test Text11  2021-09-22   757.52   
2          896 2021-09-20   456.52  Get Test Text10  2021-09-20   456.52   
3          765 2021-08-30  1300.45   Get Test Text9  2021-08-30  1200.45   
4          876 2021-09-17   786.52   Get test Text8  2021-09-17   786.52   
5          654 2021-09-18   567.52   Get Test Text7  2021-09-18   567.52   
6          522 2021-09-19  3457.04   Get Test Text6  2021-09-19  3457.04   
7          754 2021-09-21  7635.52     Get Test Te5  2021-09-21  7635.52   
8          543 2021-09-22   756.52   Get Test Text4  2021-09-22   756.52   
9          431 2021-09-19  4567.52   Get Test Text3  2021-09-19  4567.52   
10         421 2021-09-20  4566.02   Get Test Text2  2021-09-20  4566.02   

               comm  AMT_RES  DT_RES  Comm_RES  
0    Get Test Text1     True   False  

In [63]:
#4. Выгрузка в файл
result_folder = f'/home/ab170692gda'

data_result[['TranID_CSV'
            ,'DT_RES'
            ,'AMT_RES'
            ,'Comm_RES'
            ]].to_csv(f'{result_folder}/resultData.csv'
            ,sep=","
            ,decimal='.'
            # ,header=False
            ,index=False)  